# LinkedIn Queen Puzzle

## What is queen puzzle?


It is yet again, another board game puzzles by LinkedIn. Everyday you get a board, that the cells are colored.

The rules are simple:
- There should be one and only one queen in every row and column.
- There should be one and only one queen for each color region,
- Queens cannot be in the cells next to each other. Not even diagonal cells.

Here is an example board:

![Example 1](./images/sample_1.png)

Let's see how we can solve this puzzle, using optimization and OR-Tools

In [1]:
[
    ["p", "p", "p", "p", "o", "b", "b", "b"],
    ["p", "p", "p", "o", "o", "b", "b", "b", ],
    ["p", "p", "p", "p", "o", "o", "b", "b", ],
    ["p", "g", "p", "p", "p", "gr", "gr", "gr", ],
    ["r", "g", "g", "p", "p", "p", "gr", "p", ],
    ["r", "g", "y", "y", "p", "p", "p", "p", ],
    ["r", "dg", "dg", "y", "y", "p", "p", "p", ],
    ["r", "dg", "fg", "dg", "p", "p", "p", "p", ],
]

[['p', 'p', 'p', 'p', 'o', 'b', 'b', 'b'],
 ['p', 'p', 'p', 'o', 'o', 'b', 'b', 'b'],
 ['p', 'p', 'p', 'p', 'o', 'o', 'b', 'b'],
 ['p', 'g', 'p', 'p', 'p', 'gr', 'gr', 'gr'],
 ['r', 'g', 'g', 'p', 'p', 'p', 'gr', 'p'],
 ['r', 'g', 'y', 'y', 'p', 'p', 'p', 'p'],
 ['r', 'dg', 'dg', 'y', 'y', 'p', 'p', 'p'],
 ['r', 'dg', 'fg', 'dg', 'p', 'p', 'p', 'p']]

## Importing required packages

In [2]:
from dataclasses import dataclass
from tabulate import tabulate
from ortools.sat.python.cp_model import CpModel, IntVar, CpSolver, OPTIMAL

## Queen Solver

In [3]:
class QueenSolver:
    def __init__(self, cell_colors:list[list[str]]):
        if len(cell_colors) != len(cell_colors[0]):
            raise ValueError("Expecting a square board.")

        self.n = len(cell_colors)
        if any(len(row) != self.n for row in cell_colors):
            raise ValueError("Expecting a square board with all the same size rows.")

        self.cell_colors = cell_colors

        self.model = None
        self.variables = None

    def get_neighbors(self, row: int, col: int) -> list[tuple[int, int]]:
        return [
            (row + di, col + dj)
            for di in [-1, 0, 1]
            for dj in [-1, 0, 1]
            if (
                ((di,dj) != (0,0)) &
                (col + dj >=0) &
                (col + dj < self.n) &
                (row + di >=0) &
                (row + di < self.n)
            )
        ]
        
    def _prepare(self):
        self.model = CpModel()

        self.variables = [
            [
                self.model.NewBoolVar(f"{i}:{j}")
                for j in range(self.n)
            ]
            for i in range(self.n)
        ]

        # one and only one queen per rows
        for i in range(self.n):
            self.model.Add(
                sum(self.variables[i]) == 1
            )

        # one and only one queen per columns
        for j in range(self.n):
            self.model.Add(
                sum([
                    self.variables[i][j]
                    for i in range(self.n)
                ]) == 1
            )

        # no queen next to each other; not even diagonally
        for i in range(self.n):
            for j in range(self.n):
                self.model.Add(
                    sum([
                        self.variables[neighbor_row][neighbor_col]
                        for (neighbor_row, neighbor_col) in self.get_neighbors(row=i, col=j)
                    ]) == 0
                ).OnlyEnforceIf(self.variables[i][j])

        # One queen per region
        region_map: dict[str, list[IntVar]] = {}

        for i in range(self.n):
            for j in range(self.n):
                region_map.setdefault(self.cell_colors[i][j], []).append(self.variables[i][j])

        for v in region_map.values():
            self.model.add(sum(v) == 1)

    def solve(self):
        self._prepare()
        self.solver = CpSolver()
        status = self.solver.solve(self.model)

        output = status == OPTIMAL
        if output:
            solution = [
                [
                    "Q" if self.solver.value(self.variables[row_idx][col_idx]) == 1 else "."
                    for col_idx in range(self.n)
                ]
                for row_idx in range(self.n)    
            ]

            print(tabulate(solution, tablefmt="grid"))

        return output

## Trying it out

### Exmaple 1

In [4]:
queen_solver = QueenSolver(
    cell_colors=[
        ["p", "p", "p", "p", "o", "b", "b", "b"],
        ["p", "p", "p", "o", "o", "b", "b", "b", ],
        ["p", "p", "p", "p", "o", "o", "b", "b", ],
        ["p", "g", "p", "p", "p", "gr", "gr", "gr", ],
        ["r", "g", "g", "p", "p", "p", "gr", "p", ],
        ["r", "g", "y", "y", "p", "p", "p", "p", ],
        ["r", "dg", "dg", "y", "y", "p", "p", "p", ],
        ["r", "dg", "dg", "dg", "p", "p", "p", "p", ],
    ]
)
queen_solver.solve()

+---+---+---+---+---+---+---+---+
| . | . | . | Q | . | . | . | . |
+---+---+---+---+---+---+---+---+
| . | . | . | . | . | . | . | Q |
+---+---+---+---+---+---+---+---+
| . | . | . | . | . | Q | . | . |
+---+---+---+---+---+---+---+---+
| . | Q | . | . | . | . | . | . |
+---+---+---+---+---+---+---+---+
| . | . | . | . | . | . | Q | . |
+---+---+---+---+---+---+---+---+
| Q | . | . | . | . | . | . | . |
+---+---+---+---+---+---+---+---+
| . | . | . | . | Q | . | . | . |
+---+---+---+---+---+---+---+---+
| . | . | Q | . | . | . | . | . |
+---+---+---+---+---+---+---+---+


True

### Example 2
![sample 2](./images/sample_2.png)

In [5]:
queen_solver = QueenSolver(
    cell_colors=[
        ["p", "p", "p", "p", "p", "p", "p"],
        ["p", "p", "p", "b", "g", "a", "o"],
        ["p", "p", "p", "b", "g", "a", "o"],
        ["p", "p", "p", "b", "g", "a", "o"],
        ["p", "p", "r", "b", "g", "a", "o"],
        ["p", "y", "r", "b", "g", "a", "o"],
        ["p", "y", "r", "b", "g", "a", "o"],
    ]
)
queen_solver.solve()

+---+---+---+---+---+---+---+
| Q | . | . | . | . | . | . |
+---+---+---+---+---+---+---+
| . | . | . | . | . | . | Q |
+---+---+---+---+---+---+---+
| . | . | . | Q | . | . | . |
+---+---+---+---+---+---+---+
| . | . | . | . | . | Q | . |
+---+---+---+---+---+---+---+
| . | . | Q | . | . | . | . |
+---+---+---+---+---+---+---+
| . | . | . | . | Q | . | . |
+---+---+---+---+---+---+---+
| . | Q | . | . | . | . | . |
+---+---+---+---+---+---+---+


True

## 2025-12-07

In [6]:
queen_solver = QueenSolver(
    cell_colors=[
        [ "p",  "p",  "p",  "p",  "p",  "p",  "p",  "o",  "r", ],
        ["op", "op",  "p", "op", "op",  "p",  "o",  "o",  "r", ],
        [ "p", "op", "op", "op",  "p",  "p",  "o",  "r",  "r", ],
        [ "p",  "p",  "p",  "p",  "p",  "w",  "o",  "o",  "r", ],
        [ "p",  "b",  "p",  "p",  "p",  "w",  "w",  "o",  "y", ],
        [ "p",  "b",  "b",  "p",  "w",  "w",  "w",  "y",  "y", ],
        [ "p",  "p",  "b", "dg",  "w",  "g",  "g",  "g",  "y", ],
        [ "p",  "b",  "b", "dg",  "g",  "g", "dg",  "g",  "g", ],
        [ "p",  "b", "dg", "dg", "dg", "dg", "dg", "dg", "dg", ],
    ]
)
queen_solver.solve()

+---+---+---+---+---+---+---+---+---+
| . | . | . | . | . | . | . | . | Q |
+---+---+---+---+---+---+---+---+---+
| . | . | . | . | . | . | Q | . | . |
+---+---+---+---+---+---+---+---+---+
| . | . | . | Q | . | . | . | . | . |
+---+---+---+---+---+---+---+---+---+
| . | . | . | . | . | Q | . | . | . |
+---+---+---+---+---+---+---+---+---+
| . | Q | . | . | . | . | . | . | . |
+---+---+---+---+---+---+---+---+---+
| . | . | . | . | . | . | . | Q | . |
+---+---+---+---+---+---+---+---+---+
| Q | . | . | . | . | . | . | . | . |
+---+---+---+---+---+---+---+---+---+
| . | . | . | . | Q | . | . | . | . |
+---+---+---+---+---+---+---+---+---+
| . | . | Q | . | . | . | . | . | . |
+---+---+---+---+---+---+---+---+---+


True